## MLP Final Project

Coleton Van Valkenburgh

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
%matplotlib inline
plt.style.use('bmh')

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
#VRAM_ALLOC = # in MB, up to 80_000 if you are using the A100 GPU

#if tf.config.list_physical_devices('GPU'):
    
#    for GPU in tf.config.list_physical_devices('GPU'):
        
#        tf.config.experimental.set_virtual_device_configuration(GPU,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=VRAM_ALLOC)])

In [ ]:
# Load Training Data
X = np.load('data_train_corrected_final.npy')
t = np.load('labels_train_corrected_final.npy')
X.shape,t.shape

In [ ]:
labels_names = ['Stop','Yield','Red Light','Green Light','Roundabout','Right Turn Only',
                'Do Not Enter','Crosswalk','Handicap Parking','No Parking']

In [ ]:
#Randomly partition the data into training (10%) and test sets (90%)
X_training, X_test, t_training, t_test = train_test_split(X.T, t, shuffle=True, test_size=0.9)

X_train, X_val, t_train, t_val = train_test_split(X_training, t_training, shuffle=True, test_size=0.1)

X_train.shape, X_val.shape, X_test.shape, t_train.shape, t_val.shape, t_test.shape

In [ ]:
type(X_test)

In [ ]:
# Converts the image array back into the size (N, H, W, D)
X_train = tf.constant(X_train.reshape((X_train.shape[0],300,300,3))) #Returns the images to their origial RGB configurations

X_val = tf.constant(X_val.reshape((X_val.shape[0],300,300,3))) #Returns the images to their origial RGB configurations

X_test = tf.constant(X_test.reshape((X_test.shape[0],300,300,3))) #Returns the images to their origial RGB configurations

X_train.shape, X_val.shape, X_test.shape

In [ ]:
# Hyperparameters
lr = 0.01
num_of_epochs = 5 #Number of times will loop through the data
batch_size = 64 #Determines how the dataset is divided
optim = "Adam"

In [ ]:
# CNN Model

model = keras.models.Sequential([
    keras.layers.Conv2D(32, 7, activation='relu', padding='same', input_shape=[300,300,3]),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=keras.optimizers.Adam(learning_rate=lr),
             metrics=['accuracy'])

In [ ]:
model.fit(X_train, t_train, epochs=num_of_epochs, batch_size=batch_size,
         validation_data=(X_val, t_val),
         callbacks=[keras.callbacks.EarlyStopping(patience=10),
                   keras.callbacks.ModelCheckpoint('CNN_trial_1', save_best_only=True)])

In [ ]:
#model = keras.models.load_model('CNN_trial_1')

In [ ]:
y_train = np.argmax(model.predict(X_train), axis=1)
y_test = np.argmax(model.predict(X_test), axis=1)

In [ ]:
print(classification_report(t_train, y_train, target_names=labels_names))

print(classification_report(t_test, y_test, target_names=labels_names))